In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pptk
import random
import time
from bin_2_pcd import bin2pcd

It takes in the xyz point cloud matrix generated in the previous step as input. It returns a filtered xyz point cloud matrix, without the ground points. It also returns the z coordinate of the ground, relative to the origin (ground z variable). In order to detect the ground, the assumption was made that the points describing the ground lie on a horizontal plane, within some threshold. This simplifies the problem, as the x and y coordinates of the points play no role in determining the ground, it's rather z, the height, that matters. By looking for the z coordinate with the highest number of points, one can detect the ground.

In [ ]:
def filter_ground(cloud):
    bin_range = np.arange(cloud[:,2].min(),cloud[:,2].max(), 0.1) # range between min and max of z in 0.1 step
    counts, bins = np.histogram(cloud[:,2], bins = bin_range) # find distribution of points in z-axis
    max_z = np.where(counts == max(counts))[0][0] # find for which z there is maximum points
    if max_z + 2 <= len(bins): # check that max_z + 2 won't be outside array
        ground_z = np.round(bins[max_z + 2], 2) # +2 is in order as a factor of safety and remove some outliers, if present
    else:
        ground_z = np.round(bins[-1], 2) # else just get the last bin
    objects_cloud = cloud[(cloud[:,2]>ground_z)] # remove ground points (anything below the ground plane)
    
    return objects_cloud, ground_z

Dividing the point cloud into zones is done in order to allow for an easier and more accurate clustering, and also in order to allow for more/less details to be included in the different zones. The ranges of the zones are hyperparameters that need to be chosen for the particular data. 

In [ ]:
def divide_zones(cloud, close_range, mid_range):
    dist = []
    for r in range(len(cloud)): # find the distances between each point and the origin/lidar
        radius = np.sqrt(sum(np.square(cloud[r][:2]))) # no need for z-coord, calculate dist in xy plane
        dist.append(radius) # store distances
    
    dist = np.asarray(dist)
    cloud_zone1 = cloud[dist<close_range] # limit zone 1 to just the closest points
    cloud_zone2 = cloud[(dist>close_range) & (dist<mid_range)] # zone 2 is between the two ranges
    cloud_zone3 = cloud[dist>mid_range] # zone 3 is the rest of the cloud
    if dist.max() < mid_range:
        print('Valid zone ranges required: within the cloud limits of [{},{}]'.format(dist.min(),dist.max()))
    if close_range > mid_range:
        print('Valid zone ranges required: close_range < mid_range')
        
    return cloud_zone1, cloud_zone2, cloud_zone3

This next function is the heart of the lidar clustering. It converts the xyz matrix cloud into a PCD cloud and finds the clusters using the inbuilt Open3D cluster dbscan function. It labels each point with a cluster label, including the outliers. 

The two inputs the function requires are epsilon and min points. Epsilon is the neighbourhood radius, and it determines if a point will be considered in the cluster. The larger the epsilon, the more points will be included. The min points is the minimum number of points that need to be within epsilon radius to a point, for that point to be part of the cluster. A larger number would allow only closely positioned points to cluster.

In [ ]:
def find_zone_clusters(zone_cloud, epsilon, min_points):
    zone1_pcd = o3d.geometry.PointCloud()
    zone1_pcd.points = o3d.utility.Vector3dVector(zone_cloud)
    cluster_labels = np.array(zone1_pcd.cluster_dbscan(eps=epsilon, min_points=min_points))# use DBSCAN for clustering
    
    cluster_labels_filtered = cluster_labels[cluster_labels!=-1] # remove all outliers based on cluster
    zone_cloud_filtered = zone_cloud[cluster_labels!=-1] # also remove outlier points from cloud
    return cluster_labels_filtered, zone_cloud_filtered

The function takes in a point cloud xyz matrix and its cluster labels and produces a 3D view of the data using Open3D. It creates a PCD from the xyz matrix. The function assigns a different color to each cluster by evenly dividing the color spectrum by the number of clusters. 

In [ ]:
def visualize_clusters(zone_cloud, cluster_labels):
    zone_pcd = o3d.geometry.PointCloud() # create point cloud
    zone_pcd.points = o3d.utility.Vector3dVector(zone_cloud)
    
    max_label = cluster_labels.max() # get the last number of the clusters
    colors = plt.get_cmap("tab20")(cluster_labels/(max_label if max_label > 0 else 1)) # create a different color for each cluster by splitting the 0-1 interval in n=num of clusters
    zone_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([zone_pcd])

Change the pcd file to the directory of the .pcd file. While testing the code, I got some files from the KITTI Vision Benchmark Suite, found on this link: http://www.cvlibs.net/datasets/kitti/. Depending on the LiDAR data you're working with, the epsilons and min_points for the different zones need to be adjusted in order to get the best clustering performance. In general, for objects close to the source, for all of the data that I tested, it's better to have a smaller epsilon and larger min_points. For object clustering far from the source, a larger epsilon and smaller min_points yielded best results. 

In [ ]:
binFileName = 'file.bin'
pcd = bin2pcd(binFileName, False) # second parameter (True/False) controls whether the .bin file is saved into the working dir
pcd = o3d.io.read_point_cloud(pcd_file) # create pcd object from file
cloud_xyz = np.asarray(pcd.points) # get xyz points
o3d.visualization.draw_geometries([pcd]) # visualize point cloud

objects_cloud_xyz, ground_z = filter_ground(cloud_xyz) # filter out the ground points

object_cloud_z1, object_cloud_z2, object_cloud_z3 = divide_zones(objects_cloud_xyz, 15, 30) # split cloud into zones

# create zones and update clouds based on clustering
cluster_z1_eps = 0.5
cluster_z2_eps = 0.8
cluster_z3_eps = 3
cluster_z1_min_points = 50
cluster_z2_min_points = 40
cluster_z3_min_points = 30

cluster_labels_z1, object_cloud_z1_f = find_zone_clusters(object_cloud_z1, cluster_z1_eps, cluster_z1_min_points)
visualize_clusters(object_cloud_z1_f, cluster_labels_z1)

cluster_labels_z2, object_cloud_z2_f = find_zone_clusters(object_cloud_z2, cluster_z2_eps, cluster_z2_min_points)
visualize_clusters(object_cloud_z2_f, cluster_labels_z2)

cluster_labels_z3, object_cloud_z3_f = find_zone_clusters(object_cloud_z3, cluster_z3_eps, cluster_z3_min_points)
visualize_clusters(object_cloud_z3_f, cluster_labels_z3

In order to combine all the zones into one point cloud and display that, first we need to scale the cluster labels. For each of the three zones, the labels start from 0, so by adding the number of clusters of zone 1 to each label of zone 2, we would offset the labels. Doing the same with zone 3 (but now offsetting by the combination of zones 1 and 2), we get all unique labels. Concatenating all the point clouds and the cluster labels gives us the overall picture. 

In [ ]:
cluster_labels_z2 = cluster_labels_z2 + (cluster_labels_z1.max() + 1) # shift all labels by num of z1 labels
cluster_labels_z3 = cluster_labels_z3 + (cluster_labels_z2.max() + 1)  # shift all labels by num of new z2 labels
all_cluster_labels = np.concatenate((cluster_labels_z1, cluster_labels_z2))# , cluster_labels_z3)) # concatenate all the labels
clustered_cloud_xyz = np.concatenate((object_cloud_z1_f, object_cloud_z2_f))#, object_cloud_z3_f)) # combine all point clouds for the 3 zones

visualize_clusters(clustered_cloud_xyz, all_cluster_labels)